In [1]:
import numpy as np
from functools import cache
from itertools import zip_longest
import sys

%load_ext line_profiler

In [2]:
class Piece():
  h_edges: np.ndarray[np.uint8, np.uint8]
  v_edges: np.ndarray[np.uint8, np.uint8]
  h_height: int
  h_width: int
  v_height: int
  v_width: int

  def __init__(self, h_edges: tuple[tuple[bool]] | np.ndarray[np.uint8, np.uint8], v_edges: tuple[tuple[bool]] | np.ndarray[np.uint8, np.uint8]):
    h_edges = np.asarray(h_edges)
    v_edges = np.asarray(v_edges)

    # Trim piece to size
    while h_edges.size and ~np.any(h_edges[-1]) and v_edges.size and ~np.any(v_edges[-1]):
       h_edges = h_edges[:-1]
       v_edges = v_edges[:-1]

    while h_edges.size and ~np.any(h_edges[:,-1]) and v_edges.size and ~np.any(v_edges[:,-1]):
       h_edges = h_edges[:,:-1]
       v_edges = v_edges[:,:-1]
    
    # Edge case: when all h_edges or v_edges are trimmed from one side
    self.h_height = 0 if not h_edges.size else h_edges.shape[0]
    self.h_width = 0 if not h_edges.size else h_edges.shape[1]
    self.v_height = 0 if not v_edges.size else v_edges.shape[0]
    self.v_width = 0 if not v_edges.size else v_edges.shape[1]
    self.h_edges = np.packbits(h_edges, axis=0)
    self.v_edges = np.packbits(v_edges, axis=0)

  def __repr__(self):
    h_edges = np.unpackbits(self.h_edges, axis=0, count=self.h_height)
    v_edges = np.unpackbits(self.v_edges, axis=0, count=self.v_height)
    buffer = []
    for h_row, v_row in zip_longest(h_edges, v_edges):
        if h_row is not None:
          buffer.append(" " + " ".join('-' if h_edge else ' ' for h_edge in h_row))
        if v_row is not None:
          buffer.append(" ".join('|' if v_edge else ' ' for v_edge in v_row))
    return "\n".join(buffer)
  
  def __hash__(self):
     return hash((tuple(self.h_edges.flatten()), tuple(self.v_edges.flatten())))

  def __eq__(self, other):
      if isinstance(other, Piece):
          return np.array_equal(self.h_edges, other.h_edges) and np.array_equal(self.v_edges, other.v_edges)
      return NotImplemented
    
  def rotated(self):
    unpacked_h_edges = np.unpackbits(self.h_edges, axis=0, count=self.h_height)
    unpacked_v_edges = np.unpackbits(self.v_edges, axis=0, count=self.v_height)

    h_edges = np.rot90(unpacked_v_edges, 1)
    v_edges = np.rot90(unpacked_h_edges, 1)

    while h_edges.size and ~np.any(h_edges[0]) and v_edges.size and ~np.any(v_edges[0]):
      # top row is empty, so we can shift digit up
      h_edges = np.roll(h_edges, -1, 0)
      v_edges = np.roll(v_edges, -1, 0)

    return Piece(h_edges, v_edges)
  
  @cache
  def rotations(self):
    a = self
    b = a.rotated()
    c = b.rotated()
    d = c.rotated()
    return set((a, b, c, d))

pieces = [
  Piece(h_edges=((True, False), (True, False), (False, False)), v_edges=((True, True, False), (False, False, False))),
  Piece(h_edges=((False, False), (False, False), (False, False)), v_edges=((True, False, False), (True, False, False))),
  Piece(h_edges=((True, False), (True, False), (True, False)), v_edges=((False, True, False), (True, False, False))),
  Piece(h_edges=((True, False), (True, False), (True, False)), v_edges=((False, True, False), (False, True, False))),
  Piece(h_edges=((False, False), (True, False), (False, False)), v_edges=((True, True, False), (False, True, False))),
  Piece(h_edges=((True, False), (True, False), (True, False)), v_edges=((True, False, False), (False, True, False))),
  Piece(h_edges=((True, False), (True, False), (True, False)), v_edges=((True, False, False), (True, True, False))),
  Piece(h_edges=((True, False), (False, False), (False, False)), v_edges=((False, True, False), (False, True, False))),
  Piece(h_edges=((True, False), (True, False), (True, False)), v_edges=((True, True, False), (True, True, False))),
  Piece(h_edges=((True, False), (True, False), (True, False)), v_edges=((True, True, False), (False, True, False))),
]

print(sys.getsizeof(pieces[0].h_edges))

129


In [3]:
pieces[0]

 -
| |
 -

In [4]:
pieces[1]

|
|

In [5]:
pieces[9].rotations()

{   -
 | | |
  - -,
  -
 |  
  -
 | |
  -,
  -
 | |
  -
   |
  -,
  - -
 | | |
  -  }

In [6]:
pieces[0].rotations()

{ -
 | |
  -}

In [7]:
class Puzzle():
  H_HEIGHT = 5
  H_WIDTH = 5
  V_HEIGHT = 4
  V_WIDTH = 6
  h_edges: np.ndarray[np.uint8, np.uint8]
  v_edges: np.ndarray[np.uint8, np.uint8]

  def __init__(
      self,
      h_edges=np.packbits(np.zeros((H_HEIGHT,H_WIDTH), dtype=np.uint8), axis=0),
      v_edges=np.packbits(np.zeros((V_HEIGHT,V_WIDTH), dtype=np.uint8), axis=0),
    ):
    self.h_edges = h_edges
    self.v_edges = v_edges

  def __repr__(self):
    h_edges = np.unpackbits(self.h_edges, axis=0, count=self.H_HEIGHT)
    v_edges = np.unpackbits(self.v_edges, axis=0, count=self.V_HEIGHT)
    buffer = []  
    for h_row, v_row in zip_longest(h_edges, v_edges):
        if h_row is not None:
          buffer.append(" " + " ".join('-' if h_edge else '.' for h_edge in h_row))
        if v_row is not None:
          buffer.append(" ".join('|' if v_edge else '.' for v_edge in v_row))
    return "\n".join(buffer)

  def __hash__(self):
    return hash((tuple(self.h_edges.flatten()), tuple(self.v_edges.flatten())))

  def __eq__(self, other):
    if isinstance(other, Puzzle):
        return np.array_equal(self.h_edges, other.h_edges) and np.array_equal(self.v_edges, other.v_edges)
    return NotImplemented

  def with_piece(self, piece: Piece, v_offset: int, h_offset: int) -> "Puzzle":
    if (v_offset + piece.h_height > self.H_HEIGHT):
      return None
    if (h_offset + piece.h_width > self.H_WIDTH):
      return None
    if (v_offset + piece.v_height > self.V_HEIGHT):
      return None
    if (h_offset + piece.v_width > self.V_WIDTH):
      return None
    
    mask_h_edges = np.zeros_like(self.h_edges)
    if piece.h_edges.size:
      mask_h_edges[:,h_offset:h_offset+piece.h_edges.shape[1]] = piece.h_edges >> v_offset

    mask_v_edges = np.zeros_like(self.v_edges)
    if piece.v_edges.size:
      mask_v_edges[:,h_offset:h_offset+piece.v_edges.shape[1]] = piece.v_edges >> v_offset

    if np.any(np.bitwise_and(self.h_edges, mask_h_edges)):
      return None
    if np.any(np.bitwise_and(self.v_edges, mask_v_edges)):
      return None

    np.bitwise_or(self.h_edges, mask_h_edges, out=mask_h_edges)
    np.bitwise_or(self.v_edges, mask_v_edges, out=mask_v_edges)

    return Puzzle(mask_h_edges, mask_v_edges)
  
  def available_spaces(self):
    # find top-left corners which are not filled like |-
    h_edges = np.unpackbits(self.h_edges, axis=0, count=self.H_HEIGHT)
    v_edges = np.unpackbits(self.v_edges, axis=0, count=self.V_HEIGHT)

    v_offset = 0
    for h_row, v_row in zip_longest(h_edges, v_edges):
      if h_row is None:
        h_row = (1,) * self.H_WIDTH
      if v_row is None:
        v_row = (1,) * self.V_WIDTH
      
      h_offset = 0
      for h_edge, v_edge in zip_longest(h_row, v_row):
        if not h_edge or not v_edge:
          yield (v_offset, h_offset)
        h_offset += 1
      v_offset += 1

In [8]:
%timeit Puzzle().with_piece(pieces[7], 2, 3)

48.6 µs ± 12.9 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [9]:
%lprun -f Puzzle.with_piece Puzzle().with_piece(pieces[7], 2, 3)

Timer unit: 1e-07 s

Total time: 0.0002531 s
File: C:\Users\Vidminas\AppData\Local\Temp\ipykernel_1148\3260764882.py
Function: with_piece at line 36

Line #      Hits         Time  Per Hit   % Time  Line Contents
    36                                             def with_piece(self, piece: Piece, v_offset: int, h_offset: int) -> "Puzzle":
    37         1         36.0     36.0      1.4      if (v_offset + piece.h_height > self.H_HEIGHT):
    38                                                 return None
    39         1         11.0     11.0      0.4      if (h_offset + piece.h_width > self.H_WIDTH):
    40                                                 return None
    41         1         12.0     12.0      0.5      if (v_offset + piece.v_height > self.V_HEIGHT):
    42                                                 return None
    43         1         18.0     18.0      0.7      if (h_offset + piece.v_width > self.V_WIDTH):
    44                                                 re

In [10]:
Puzzle()

 . . . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . .

In [11]:
Puzzle().with_piece(pieces[0], 3, 4)

 . . . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . -
. . . . | |
 . . . . -

In [12]:
Puzzle().with_piece(pieces[7].rotated(), 0, 0)

 - - . . .
| . . . . .
 . . . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . .

In [13]:
Puzzle().with_piece(pieces[3], 0, 0).with_piece(pieces[4].rotated(), 0, 1)

 - - - . .
. | | . . .
 - - . . .
. | . . . .
 - . . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . .

In [14]:
Puzzle().with_piece(pieces[9].rotated().rotated().rotated(), 1, 0)

 . . . . .
. . . . . .
 . - . . .
| | | . . .
 - - . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . .

In [15]:
emptys = tuple(Puzzle().available_spaces())
print(emptys)
assert len(emptys) == max(Puzzle.H_HEIGHT, Puzzle.V_HEIGHT) * max(Puzzle.H_WIDTH, Puzzle.V_WIDTH)

((0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5))


In [16]:
emptys = tuple(Puzzle().with_piece(pieces[8], 1, 0).available_spaces())
print(emptys)
assert (1, 0) not in emptys
assert (2, 0) not in emptys
assert len(emptys) == max(Puzzle.H_HEIGHT, Puzzle.V_HEIGHT) * max(Puzzle.H_WIDTH, Puzzle.V_WIDTH) - 2

((0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5))


In [17]:
# larger pieces ordered first, as this prunes more candidate solutions
pieces_order = (8, 9, 6, 5, 2, 0, 4, 3, 7, 1)
bad_puzzles = set()

# pick a puzzle piece
# choose rotation
# find top-left free corner
# put piece if it fits
# check if solved
def attempt(puzzle: Puzzle, used_pieces: dict[int, tuple]):
  if puzzle in bad_puzzles:
    return None
  if len(used_pieces) == len(pieces):
    return puzzle, used_pieces
  
  available_spaces = tuple(puzzle.available_spaces())
  
  for i in pieces_order:
    if i in used_pieces:
      continue

    for rotation in pieces[i].rotations():
      for v_offset, h_offset in available_spaces:
        candidate = puzzle.with_piece(rotation, v_offset, h_offset)
  
        if candidate is not None:
          if candidate in bad_puzzles:
            continue

          candidate_used_pieces = used_pieces.copy()
          candidate_used_pieces[i] = (rotation, v_offset, h_offset)
          result = attempt(candidate, candidate_used_pieces)
          if result is not None:
            return result

  bad_puzzles.add(puzzle)
  return None

In [18]:
# A blank start does not have a unique solution, so this is just for profiling
# Interrupt manually after it runs for about a minute
%lprun -f attempt attempt(Puzzle(), {})

*** KeyboardInterrupt exception caught in code being profiled.

Timer unit: 1e-07 s

Total time: 57.5152 s
File: C:\Users\Vidminas\AppData\Local\Temp\ipykernel_1148\3777802545.py
Function: attempt at line 10

Line #      Hits         Time  Per Hit   % Time  Line Contents
    10                                           def attempt(puzzle: Puzzle, used_pieces: dict[int, tuple]):
    11      8653    1105502.0    127.8      0.2    if puzzle in bad_puzzles:
    12                                               return None
    13      8653     232807.0     26.9      0.0    if len(used_pieces) == len(pieces):
    14                                               return puzzle, used_pieces
    15                                             
    16      8653   11045109.0   1276.4      1.9    available_spaces = tuple(puzzle.available_spaces())
    17                                             
    18     95138     643218.0      6.8      0.1    for i in pieces_order:
    19     86493     620941.0      7.2      0.1      if i in used_pieces:
    20     58989   

In [19]:
# Puzzle 1 - Starter
bad_puzzles = set()
puzzle1_config = {
  1: (pieces[1], 2, 0),
  2: (pieces[2].rotated(), 1, 2),
  3: (pieces[3].rotated().rotated(), 2, 1),
  4: (pieces[4].rotated(), 0, 1),
  6: (pieces[6].rotated(), 3, 2),
  7: (pieces[7], 1, 4),
  9: (pieces[9].rotated(), 0, 3),
}
puzzle1 = Puzzle()
for starting_piece, v_offset, h_offset in puzzle1_config.values():
  puzzle1 = puzzle1.with_piece(starting_piece, v_offset, h_offset)
print(puzzle1)
print()

solution1 = attempt(puzzle1, puzzle1_config)
assert solution1 is not None
print(solution1[0])
assert solution1[1][8] == (pieces[8], 0, 0)
assert solution1[1][5] == (pieces[5], 2, 4)
assert solution1[1][0] == (pieces[0], 2, 2)

 . - - - -
. . | | | |
 . - - - -
. . | | | |
 . - . - .
| | . . . |
 . - . - .
| | | | | .
 . - - - .

 - - - - -
| | | | | |
 - - - - -
| | | | | |
 - - - - -
| | | | | |
 . - - - -
| | | | | |
 . - - - -


In [20]:
# Puzzle 2 - Starter
bad_puzzles = set()
puzzle2_config = {
  8: (pieces[8], 0, 0),
  4: (pieces[4].rotated(), 0, 1),
  3: (pieces[3].rotated(), 0, 3),
  9: (pieces[9].rotated().rotated().rotated(), 2, 0),
  5: (pieces[5], 1, 2),
  6: (pieces[6].rotated().rotated().rotated(), 1, 3),
  0: (pieces[0], 2, 4),
}
puzzle2 = Puzzle()
for starting_piece, v_offset, h_offset in puzzle2_config.values():
  puzzle2 = puzzle2.with_piece(starting_piece, v_offset, h_offset)
print(puzzle2)
print()

solution2 = attempt(puzzle2, puzzle2_config)
assert solution2 is not None
print(solution2[0])
assert solution2[1][7] == (pieces[7].rotated().rotated().rotated(), 3, 0)
assert solution2[1][1] == (pieces[1].rotated(), 4, 2)
assert solution2[1][2] == (pieces[2].rotated(), 3, 3)

 - - - - -
| | | | | |
 - - - - -
| | | | | |
 - - - - -
| | | | | |
 - - - . -
. . . . . .
 . . . . .

 - - - - -
| | | | | |
 - - - - -
| | | | | |
 - - - - -
| | | | | |
 - - - - -
. . | | | |
 - - - - -


In [21]:
# Puzzle 10 - Starter
bad_puzzles = set()
puzzle10_config = {
  6: (pieces[6], 2, 0),
  9: (pieces[9].rotated(), 2, 1),
  4: (pieces[4].rotated().rotated().rotated(), 3, 1),
  0: (pieces[0], 3, 3),
  5: (pieces[5], 2, 4),
}
puzzle10 = Puzzle()
for starting_piece, v_offset, h_offset in puzzle10_config.values():
  puzzle10 = puzzle10.with_piece(starting_piece, v_offset, h_offset)
print(puzzle10)
print()

solution10 = attempt(puzzle10, puzzle10_config)
assert solution10 is not None
print(solution10[0])
assert solution10[1][3] == (pieces[3].rotated(), 0, 0)
assert solution10[1][8] == (pieces[8].rotated(), 0, 3)
assert solution10[1][7] == (pieces[7].rotated(), 1, 0)
assert solution10[1][2] == (pieces[2].rotated(), 1, 2)
assert solution10[1][1] == (pieces[1], 1, 5)

 . . . . .
. . . . . .
 . . . . .
. . . . . .
 - - - . -
| | | | | .
 - - - - -
| | | | | |
 - - - - -



 - - . - -
| | | | | |
 - - - - -
| . | | | |
 - - - - -
| | | | | |
 - - - - -
| | | | | |
 - - - - -


In [22]:
# Puzzle 16 - Starter
bad_puzzles = set()
puzzle16_config = {
  2: (pieces[2], 2, 0),
  8: (pieces[8].rotated(), 2, 2),
  9: (pieces[9].rotated().rotated().rotated(), 0, 3),
}
puzzle16 = Puzzle()
for starting_piece, v_offset, h_offset in puzzle16_config.values():
  puzzle16 = puzzle16.with_piece(starting_piece, v_offset, h_offset)
print(puzzle16)
print()

solution16 = attempt(puzzle16, puzzle16_config)
assert solution16 is not None
print(solution16[0])
assert solution16[1][3] == (pieces[3].rotated(), 0, 0)
assert solution16[1][1] == (pieces[1].rotated(), 0, 2)
assert solution16[1][7] == (pieces[7].rotated(), 1, 0)
assert solution16[1][5] == (pieces[5].rotated(), 1, 1)
assert solution16[1][4] == (pieces[4], 1, 4)
assert solution16[1][0] == (pieces[0], 3, 1)
assert solution16[1][6] == (pieces[6].rotated(), 3, 3)

 . . . . -
. . . | | |
 . . . - -
. . . . . .
 - . - - .
. | | | | .
 - . - - .
| . . . . .
 - . . . .

 - - - - -
| | | | | |
 - - - - -
| | | | | |
 - - - - -
. | | | | |
 - - - - -
| | | | | |
 - - . - -


In [23]:
# Puzzle 112 - Master
bad_puzzles = set()
puzzle112_config = {
  6: (pieces[6], 0, 0),
  3: (pieces[3].rotated(), 0, 3),
  4: (pieces[4].rotated(), 3, 0),
  5: (pieces[5], 2, 4)
}
puzzle112 = Puzzle()
for starting_piece, v_offset, h_offset in puzzle112_config.values():
  puzzle112 = puzzle112.with_piece(starting_piece, v_offset, h_offset)
print(puzzle112)
print()

solution112 = attempt(puzzle112, puzzle112_config)
assert solution112 is not None
print(solution112[0])
assert solution112[1][0] == (pieces[0], 0, 1)
assert solution112[1][8] == (pieces[8].rotated(), 1, 2)
assert solution112[1][1] == (pieces[1], 2, 0)
assert solution112[1][2] == (pieces[2].rotated(), 2, 1)
assert solution112[1][7] == (pieces[7], 1, 4)
assert solution112[1][9] == (pieces[9].rotated().rotated().rotated(), 3, 2)

 - . . - -
| . . | | |
 - . . . .
| | . . . .
 - . . . -
. . . . | .
 - - . . -
. | . . . |
 - . . . -

 - - . - -
| | | | | |
 - - - - -
| | | | | |
 - - - - -
| | | | | |
 - - - - -
| | | | | |
 - . - - -


In [24]:
# Puzzle 120 - Master
bad_puzzles = set()
rot9 = pieces[9].rotated().rotated().rotated()
puzzle120 = Puzzle().with_piece(rot9, 1, 0)
print(puzzle120)
print()

solution120 = attempt(puzzle120, {9: (rot9, 1, 0)})
assert solution120 is not None
print(solution120[0])
# total bad solutions = 11115514

 . . . . .
. . . . . .
 . - . . .
| | | . . .
 - - . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . .



KeyboardInterrupt: 

In [25]:
len(bad_puzzles)

26763

In [26]:
next(iter(bad_puzzles))

 - - . - .
| | | | | .
 - - . - .
| | | | | .
 - - . - .
. . . . | |
 . - . . -
| | | . . |
 - . . - -